# Cleaning and Aspect Detection

In [120]:
import pandas as pd
import numpy as np
import spacy
import nltk
import string
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import torch
from textblob import TextBlob
from tqdm.notebook import tqdm
from symspellpy import SymSpell, Verbosity
import pkg_resources
tqdm.pandas()
pd.set_option('display.max_colwidth', None)
from pyabsa import ATEPCCheckpointManager
from collections import defaultdict
import pandas as pd
import ast, re, itertools
from pathlib import Path
from sklearn.metrics import f1_score, precision_recall_fscore_support, accuracy_score



## Goal:
In this file we perform some data cleaning of our review data, we compare how applying pyabsa to different 

In [121]:
df = pd.read_csv('filtered_england_recent.csv')

## Data sampling and cleaning
Working with a sample size of 1% to test cleaning techniques before applying pyabsa

In [122]:
# Load SpaCy English model
nlp = spacy.load("en_core_web_sm")

# Sample 5% of the data 
df_sample = df.sample(frac=0.01, random_state=21).copy()
def clean_min(text):
    if not isinstance(text, str):
        return ""
    return text.replace('\n', ' ').replace('\r', '').encode("ascii", errors="ignore").decode().strip()

#only lematizing, want to keep as many words as possible. 
def preprocess_text_lemmaonly(text):
    if pd.isnull(text):
        return ""
    doc = nlp(text.lower())
    return " ".join(token.lemma_ for token in doc if not token.is_punct and not token.is_space)

def preprocess_text_with_stopwords(text):
    if pd.isnull(text):
        return ""
    doc = nlp(text.lower())
    return " ".join(
        token.lemma_ 
        for token in doc 
        if not token.is_punct 
        and not token.is_space 
        and not token.is_stop
    )

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
def correct_token(token_text):
    suggestions = sym_spell.lookup(token_text, Verbosity.CLOSEST, max_edit_distance=2)
    if suggestions:
        return suggestions[0].term
    return token_text

def preprocess_text_symspell(text):
    if pd.isnull(text):
        return ""
    corrected_tokens = []
    for token in nlp(text.lower()):
        if token.is_punct or token.is_space or token.like_num or token.is_stop or len(token) < 3:
            continue
        corrected = correct_token(token.text)
        doc = nlp(corrected)  # lemmatize after correction
        for sub_token in doc:
            if not sub_token.is_punct and not sub_token.is_space:
                corrected_tokens.append(sub_token.lemma_)
    return " ".join(corrected_tokens)


# Apply preprocessing with progress bar
for col in ['pros', 'cons']:
    print(f"Processing column: {col}")
    df_sample[f'{col}_clean_min'] = df_sample[col].progress_apply(clean_min)
    df_sample[f'{col}_clean_lemma_only'] = df_sample[col].progress_apply(preprocess_text_lemmaonly)
    df_sample[f'{col}_clean_with_stopwords'] = df_sample[col].progress_apply(preprocess_text_with_stopwords)
    df_sample[f'{col}_clean_with_symspell'] = df_sample[col].progress_apply(preprocess_text_symspell)


# Preview result
df_sample.head()

Processing column: pros


  0%|          | 0/761 [00:00<?, ?it/s]

  0%|          | 0/761 [00:00<?, ?it/s]

  0%|          | 0/761 [00:00<?, ?it/s]

  0%|          | 0/761 [00:00<?, ?it/s]

...

  0%|          | 0/761 [00:00<?, ?it/s]

  0%|          | 0/761 [00:00<?, ?it/s]

  0%|          | 0/761 [00:00<?, ?it/s]

  0%|          | 0/761 [00:00<?, ?it/s]

,Unnamed: 0,firm,date_review,job_title,current,location,overall_rating,work_life_balance,culture_values,diversity_inclusion,...,cons,year,pros_clean_min,pros_clean_lemma_only,pros_clean_with_stopwords,pros_clean_with_symspell,cons_clean_min,cons_clean_lemma_only,cons_clean_with_stopwords,cons_clean_with_symspell
72228,819748,Vodafone,2021-05-04,Business Analyst,"Current Employee, more than 1 year","Reading, England, England",4,NaN,NaN,NaN,...,Pay could be a little higher,2021,Champions team work and builds a great network,champion team work and build a great network,champion team work build great network,champion team work build great network,Pay could be a little higher,pay could be a little high,pay little high,pay little higher
37518,323899,Hastings Direct,2019-05-31,Customer Service Representative,"Current Employee, more than 3 years","Bexhill, England, England",2,2.0,2.0,NaN,...,"The company is compiled of 90% student employees, however, team leaders at Hastings, do not remember this, and it is an ordeal for students to get time off for exams If a student needs time off for exams and they have the holiday time available management should not require them to make up the time, they should have the option to use a holiday. The company should not be allowed to deny any student time off for exams. I have been there for three years and I notice the changes; now it is a pressure cooker. The original TL I had recently quit because the direction the company is going in is not about treating employees fairly. The TL's have their favourites and if you are not one of them you will be subjected to bullying. Upper management does nothing to intervene and you are treated like just a number, and your only purpose here is to make as much money for the company as possible and spend less time on the phone trying to help customers. Different information told from coaches so customers are angry. Some TL's have inappropriate relationships with team members. No chance for advancement in the company unless you are friends with TL's\r\n\r\nShow Less\r\nAdvice to Management",2019,Pay and the other employees,pay and the other employee,pay employee,pay employee,"The company is compiled of 90% student employees, however, team leaders at Hastings, do not remember this, and it is an ordeal for students to get time off for exams If a student needs time off for exams and they have the holiday time available management should not require them to make up the time, they should have the option to use a holiday. The company should not be allowed to deny any student time off for exams. I have been there for three years and I notice the changes; now it is a pressure cooker. The original TL I had recently quit because the direction the company is going in is not about treating employees fairly. The TL's have their favourites and if you are not one of them you will be subjected to bullying. Upper management does nothing to intervene and you are treated like just a number, and your only purpose here is to make as much money for the company as possible and spend less time on the phone trying to help customers. Different information told from coaches so customers are angry. Some TL's have inappropriate relationships with team members. No chance for advancement in the company unless you are friends with TL's Show Less Advice to Management",the company be compile of 90 student employee however team leader at hasting do not remember this and it be an ordeal for student to get time off for exam if a student need time off for exam and they have the holiday time available management should not require they to make up the time they should have the option to use a holiday the company should not be allow to deny any student time off for exam I have be there for three year and I notice the change now it be a pressure cooker the original tl I have recently quit because the direction the company be go in be not about treat employee fairly the tl 's have their favourite and if y

In [123]:
df_sample[['pros', 'cons', 'pros_clean_lemma_only', 'pros_clean_with_stopwords', 'pros_clean_with_symspell', 'cons_clean_lemma_only','cons_clean_with_stopwords', 'cons_clean_with_symspell']].head()

,pros,cons,pros_clean_lemma_only,pros_clean_with_stopwords,pros_clean_with_symspell,cons_clean_lemma_only,cons_clean_with_stopwords,cons_clean_with_symspell
72228,Champions team work and builds a great network,Pay could be a little higher,champion team work and build a great network,champion team work build great network,champion team work build great network,pay could be a little high,pay little high,pay little higher
37518,Pay and the other employees,"The company is compiled of 90% student employees, however, team leaders at Hastings, do not remember this, and it is an ordeal for students to get time off for exams If a student needs time off for exams and they have the holiday time available management should not require them to make up the time, they should have the option to use a holiday. The company should not be allowed to deny any student time off for exams. I have been there for three years and I notice the changes; now it is a pressure cooker. The original TL I had recently quit because the direction the company is going in is not about treating employees fairly. The TL's have their favourites and if you are not one of them you will be subjected to bullying. Upper management does nothing to intervene and you are treated like just a number, and your only purpose here is to make as much money for the company as possible and spend less time on the phone trying to help customers. Different information told from coaches so customers are angry. Some TL's have inappropriate relationships with team members. No chance for advancement in the company unless you are friends with TL's\r\n\r\nShow Less\r\nAdvice to Management",pay and the other employee,pay employee,pay employee,the company be compile of 90 student employee however team leader at hasting do not remember this and it be an ordeal for student to get time off for exam if a student need time off for exam and they have the holiday time available management should not require they to make up the time they should have the option to use a holiday the company should not be allow to deny any student time off for exam I have be there for three year and I notice the change now it be a pressure cooker the original tl I have recently quit because the direction the company be go in be not about treat employee fairly the tl 's have their favourite and if you be not one of they you will be subject to bully upper management do nothing to intervene and you be treat like just a number and your only purpose here be to make as much money for the company as possible and spend less time on the phone try to help customer different information tell from coach so customer be angry some tl 's have inappropriate relationship with team member no chance for advancement in the company unless you be friend with tl 's show less advice to management,company compile 90 student employee team leader hasting remember ordeal student time exam student need time exam holiday time available management require time option use holiday company allow deny student time exam year notice change pressure cooker original tl recently quit direction company go treat employee fairly tl favourite subject bully upper management intervene treat like number purpose money company possible spend time phone try help customer different information tell coach customer angry tl inappropriate relationship team member chance advancement company friend tl advice management,company compile student employee team leader hasting remember ordeal student time exam student need time exam holiday time available management require time option use holiday company allow deny student time exam year notice change pressure cooker original recently quit direction company go treat employee fairly favourite subject bully upper management intervene treat like number purpose money company possible spend time phone try help customer different information tell coach customer angry inappropriate relationship team member chance advancement company friend

## ABSA

In [124]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [125]:
def get_aspect_sentiment(text, aspect, tone):
    if pd.isnull(text):
        return None

    # Compose prompt-like input for aspect analysis
    if tone == 'pos':
        combined = f"The following is a positive experience about {aspect}: {text}"
    else:
        combined = f"The following is a negative experience about {aspect}: {text}"

    inputs = tokenizer(combined, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        logits = model(**inputs).logits
    scores = softmax(logits.numpy()[0])

    # This model returns scores for: [1 star, 2 stars, 3 stars, 4 stars, 5 stars]
    star_labels = [1, 2, 3, 4, 5]
    result = dict(zip(star_labels, scores))
    result["label"] = star_labels[scores.argmax()]
    return result


In [126]:
from collections import Counter
failed_sentiment = Counter()

def analyze_all_aspect_groups(text, aspect_groups, tone):
    results = {}
    for aspect, keywords in aspect_groups.items():
        if any(kw in text.lower() for kw in keywords):
            sent = get_aspect_sentiment(text, aspect, tone)
            if sent:
                results[aspect] = sent
            else:
                failed_sentiment[aspect] += 1  # Track failures
    return results


## Aspect detection
Using automatic aspect detection to find what aspects get mentioned the most in the reviews

In [127]:
# Load ABSA model
aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(
    checkpoint='english',
    auto_device=True
)

# Sample for ABSA (adjust size as needed)
texts_pros = df_sample['pros_clean_min'].dropna().sample(frac=0.5, random_state=42).tolist()
texts_cons = df_sample['cons_clean_min'].dropna().sample(frac=0.5, random_state=42).tolist()

# Extract aspects from pros and cons
absa_results_pros = aspect_extractor.extract_aspect(inference_source=texts_pros, print_result=False)
absa_results_cons = aspect_extractor.extract_aspect(inference_source=texts_cons, print_result=False)

# Collect discovered aspects
from collections import Counter

aspect_counter = Counter()
for result in absa_results_pros + absa_results_cons:
    for asp in result.get("aspect", []):
        aspect_counter[asp.lower()] += 1

# Show top discovered aspects
top_aspects = [asp for asp, count in aspect_counter.items() if count >= 3]
print(f"Top discovered aspects: {top_aspects}")


[2025-05-09 13:53:30] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-05-09 13:53:30] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-05-09 13:53:30] (2.4.1.post1) Downloading checkpoint:english 
[2025-05-09 13:53:30] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2025-05-09 13:53:30] (2.4.1.post1) Checkpoint already downloaded, skip
[2025-05-09 13:53:30] (2.4.1.post1) Load aspect extractor from checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43
[2025-05-09 13:53:30] (2.4.1.post1) config: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.config
[2025-05-09 13:53:30] (2.4.1.post1) state_dict: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apc

/opt/anaconda3/lib/python3.12/site-packages/pyabsa/tasks/AspectTermExtraction/prediction/aspect_extractor.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(
/

[2025-05-09 13:55:07] (2.4.1.post1) The results of aspect term extraction have been saved in /Users/monikakaczorowska/Desktop/glassdoor_reviews/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json


classifying aspect sentiments: 100%|██████████| 12/12 [00:44<00:00,  3.72s/it]


[2025-05-09 13:56:36] (2.4.1.post1) The results of aspect term extraction have been saved in /Users/monikakaczorowska/Desktop/glassdoor_reviews/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
Top discovered aspects: ['pay', 'projects', 'environment', 'people', 'organisation', 'company', 'colleagues', 'work', 'management', 'place', 'office', 'atmosphere', 'training', 'team', 'bonus', 'benefits', 'culture', 'employees', 'managers', 'communication', 'hours', 'staff', 'salary', 'manager', 'job', 'development', 'food', 'events', 'location', 'values', 'workload', 'facilities', 'pension', 'time', 'support', 'opportunities', 'learn', 'flexibility', 'teams', 'benefits package', 'perks', 'bonuses', 'networking', 'diversity', 'department', 'service', 'progression', 'it', 'bureaucracy', 'politics', 'processes']


In [128]:
from collections import Counter

# Combine pros and cons into one string per row
combined_texts = (
    df_sample['pros_clean_min'].fillna('') + ' ' + df_sample['cons_clean_min'].fillna('')
).str.lower()

# Count how many unique reviews mention each aspect
aspect_review_counts = {asp: combined_texts.str.contains(rf'\b{asp.lower()}\b').sum() for asp in top_aspects}

# Display sorted
print("\n🧾 Number of Reviews Mentioning Each Auto-Detected Aspect (Unique Mentions Only):")
for asp, count in sorted(aspect_review_counts.items(), key=lambda x: -x[1]):
    print(f"{asp:<15}: {count}")



🧾 Number of Reviews Mentioning Each Auto-Detected Aspect (Unique Mentions Only):
work           : 325
people         : 212
company        : 158
it             : 147
management     : 143
hours          : 122
pay            : 114
opportunities  : 106
staff          : 104
team           : 97
environment    : 90
benefits       : 89
culture        : 86
training       : 78
salary         : 75
progression    : 74
job            : 67
time           : 67
support        : 54
colleagues     : 50
place          : 50
managers       : 50
development    : 50
office         : 45
employees      : 45
organisation   : 32
bonus          : 32
manager        : 30
projects       : 25
teams          : 25
learn          : 24
politics       : 21
processes      : 21
location       : 20
perks          : 19
service        : 19
atmosphere     : 18
events         : 18
flexibility    : 17
values         : 16
communication  : 13
pension        : 13
food           : 12
workload       : 12
bureaucracy    : 12
bonuses  

## Manual aspects
Came up with a list of aspects candidates may value

In [129]:
aspect_groups = {
    "pay": ["pay", "salary", "compensation", "wage", "income", "bonus"],
    "management": ["management", "managers", "leadership", "boss", "supervisor"],
    "work-life balance": ["work-life balance", "balance", "working hours", "hours", "flexibility", "overtime", "holiday"],
    "growth": ["growth", "grow", "career", "promotion", "advancement", "development", "opportunity", "progress"],
    "stress & pressure": ["stress", "pressure", "burnout", "demanding", "high pressure", "overload", "long hours", "stressful"],
    "culture": ["culture", "environment", "atmosphere", "company culture", "values"],
    "diversity & inclusion": ["diversity", "inclusion", "equality", "equity", "inclusive", "minority", "belonging"],
    "travel": ["travel", "commute", "commuting", "relocation", "mobility"],
    "benefits": ["benefits", "perks", "healthcare", "insurance", "pension", "rewards", "gym", "lunch", "dental"],
    "remote work": ["remote", "work from home", "wfh", "home office", "hybrid", "telework"],
    "flexibility": ["flexible", "flexibility", "adjustable schedule", "shift", "adaptable"],
    "people": ["people", "colleagues", "collegues", "coworker", "coworkers", "community", "teammates"],
    "training & onboarding": ["training", "onboarding", "orientation", "new hire", "introduction", "induction"],
    "job role clarity": ["role", "responsibility", "job description", "tasks", "duties", "expectation", "scope"],
    "tools & resources": ["tools", "resources", "equipment", "software", "support", "access", "tech stack", "technology"],
    "performance feedback": ["feedback", "review", "appraisal", "evaluation", "performance", "supervision", "goals"],
    "job security": ["job security", "layoff", "firing", "termination", "redundancy", "stable job"],
    "teamwork & collaboration": ["team", "teamwork", "collaboration", "supportive", "helpful", "team spirit", "group"]
}
def safe_analyze(text, aspect_groups, tone):
    try:
        return analyze_all_aspect_groups(text, aspect_groups, tone)
    except Exception as e:
        print(f"Failed on: {repr(text)}\nError: {e}")
        return {}

df_sample['pro_aspect_sentiments'] = df_sample['pros_clean_min'].progress_apply(lambda x: analyze_all_aspect_groups(x, aspect_groups, tone='pos'))
df_sample['con_aspect_sentiments'] = df_sample['cons_clean_min'].progress_apply(lambda x: analyze_all_aspect_groups(x, aspect_groups, tone='neg'))

  0%|          | 0/761 [00:00<?, ?it/s]

  0%|          | 0/761 [00:00<?, ?it/s]

In [130]:
# Full list of all aspect labels from your aspect_groups
all_aspects = list(aspect_groups.keys())
pro_counts= Counter()
con_counts = Counter()
pro_counts_clean = Counter()
con_counts_clean = Counter()

for row in df_sample['pro_aspect_sentiments'].dropna():
    pro_counts.update(row.keys())
for row in df_sample['con_aspect_sentiments'].dropna():
    con_counts.update(row.keys())

# Ensure all aspects appear, even if 0
pro_data = [(aspect, pro_counts.get(aspect, 0)) for aspect in all_aspects]
con_data = [(aspect, con_counts.get(aspect, 0)) for aspect in all_aspects]

# Sort by count
pro_data.sort(key=lambda x: x[1])
con_data.sort(key=lambda x: x[1])

# Ensure all aspects appear, even if 0
pro_data_clean = [(aspect, pro_counts_clean.get(aspect, 0)) for aspect in all_aspects]
con_data_clean = [(aspect, con_counts_clean.get(aspect, 0)) for aspect in all_aspects]

# Sort by count
pro_data_clean.sort(key=lambda x: x[1])
con_data_clean.sort(key=lambda x: x[1])

## Ultimate aspect selection
after running automatic aspect detection, we grouped aspects into categories based on results and our own knowledge of company aspects that are important to employees and candidates

In [131]:
# 1. Normalise everything to lowercase
existing_keys = {k.lower() for k in aspect_groups.keys()}

# flatten every keyword in the dictionary
existing_keywords = {kw.lower() for kws in aspect_groups.values() for kw in kws}

# 2. Compare
# a) aspects that are missing entirely
missing_new_keys = [asp for asp in top_aspects
                    if asp.lower() not in existing_keys
                    and asp.lower() not in existing_keywords]

# b) aspects that already exist as *keywords* but not as a top‑level key
present_as_keyword = [asp for asp in top_aspects
                      if asp.lower() not in existing_keys    # not a key
                      and asp.lower() in existing_keywords]  # but inside values

print("🔍 Candidates to ADD as new keys:")
for asp in missing_new_keys:
    print("  •", asp)

print("\n✅ Already present as a keyword (maybe promote to key?):")
for asp in present_as_keyword:
    print("  •", asp)

🔍 Candidates to ADD as new keys:
  • projects
  • organisation
  • company
  • work
  • place
  • office
  • employees
  • communication
  • staff
  • manager
  • job
  • food
  • events
  • location
  • workload
  • facilities
  • time
  • opportunities
  • learn
  • teams
  • benefits package
  • bonuses
  • networking
  • department
  • service
  • progression
  • it
  • bureaucracy
  • politics
  • processes

✅ Already present as a keyword (maybe promote to key?):
  • environment
  • colleagues
  • atmosphere
  • training
  • team
  • bonus
  • managers
  • hours
  • salary
  • development
  • values
  • pension
  • support
  • perks
  • diversity


### Combining for ultimate dictionary:

In [132]:
modified_aspect_groups = {
    'perks & benefits': [
        'benefit', 'benefits', 'employee benefits', 'perks', 'health',
        'healthcare', 'insurance', 'pension', 'pension scheme', 'rewards',
        'gym', 'lunch', 'dental', 'benefits package', 'package', 'nhs',
        'stock', 'discount', 'discounts', 'food', 'coffee', 'drink',
        'drinks', 'meal', 'meals'],

    'culture': [
        'culture', 'family', 'environment', 'atmosphere', 'company culture',
        'values', 'enviroment', 'work environment', 'office environment',
        'morale', 'communication', 'communications', 'vision', 'sustainability''attitude',
        'bullying', 'interaction', 'gossip', 'dress code'],

    'diversity & inclusion': [
        'diversity', 'inclusion', 'equality', 'equity', 'inclusive',
        'minority', 'belonging'],

    'growth & learning': [
        'growth', 'grow', 'career', 'promotion', 'promotions', 'advancement',
        'development', 'opportunity', 'progress', 'opportunities', 'progression',
        'career progression', 'learn', 'learning', 'knowledge', 'courses', 'academics',
        'academic', 'academy', 'student', 'students', 'teaching', 'university', 'internal movement'],

    'hr & recruitment': [
        'hr', 'recruitment', 'talent', 'staff', 'staff members', 'staffing',
        'policies', 'training', 'onboarding', 'orientation', 'new hire', 'introduction',
        'induction'],

    'job role clarity': [
        'role', 'responsibility', 'job description', 'tasks', 'duties',
        'expectation', 'scope'],

    'job security': [
        'job security', 'security', 'safety', 'layoff', 'firing', 'termination',
        'redundancy', 'stable job', 'employment', 'staff turnover', 'turnover'],

    'management': [
        'directors', 'management', 'managers', 'leader', 'leaders',
        'leadership', 'boss', 'supervisor', 'supervisors', 'ceo', 'bosses',
        'management structure', 'management team', 'middle management',
        'senior management', 'structure', 'bureaucracy', 'bureaucratic',
        'red tape', 'tape', 'politics'],

    'office & location': [
        'office', 'office location', 'office space',
        'offices', 'location', 'locations', 'area', 'workplace',
        'building', 'buildings', 'facilities', 'space', 'room',
        'infrastructure', 'campus', 'canteen', 'parking'],

    'pay': [
        'pay', 'paid', 'money', 'salary', 'compensation', 'wage', 'wages',
        'income', 'bonus', 'bonuses', 'reward', 'salaries', 'sick pay',
        'incentives', 'pay rates', 'commission', 'contract',
        'contracts', 'terms', 'conditions'],

    'people': [
        'people', 'colleagues', 'collegues', 'coworker', 'coworkers', 'community',
        'teammates', 'social', 'crew'],

    'performance feedback': [
        'feedback', 'review', 'appraisal', 'evaluation', 'performance',
        'supervision', 'goals', 'targets', 'kpi'],

    'remote work': [
        'remote', 'work from home', 'wfh', 'home office', 'hybrid', 'telework'],

    'stress & pressure': [
        'stress', 'paced environment', 'pace', 'pressure', 'burnout',
        'demanding', 'high pressure', 'overload', 'long hours', 'stressful'],

    'teamwork & collaboration': [
        'team', 'teams', 'teamwork', 'collaboration', 'supportive',
        'helpful', 'team spirit', 'group'],

    'tools & resources': [
        'tools', 'resources', 'equipment', 'software', 'support', 'access',
        'tech stack', 'technology', 'technology', 'tech', 'innovation',
        'technologies', 'research'],

    'work-life balance': [
        'work-life balance', 'workload', 'weekends', 'balance', 'working hours',
        'hours', 'flexibility', 'overtime', 'holiday', 'life balance', 'lifestyle',
        'annual leave', 'christmas', 'flexible', 'adjustable schedule', 'shift', 'adaptable',
        'break', 'breaks']
}

### Analyzing which reviews the aspect detection works best on
comparing clean min, clean lemma only, and clean____

In [133]:
df_sample['min_pro_aspect_sentiments'] = df_sample['pros_clean_min'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='pos'))
df_sample['min_con_aspect_sentiments'] = df_sample['cons_clean_min'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='neg'))
df_sample['lemma_pro_aspect_sentiments'] = df_sample['pros_clean_lemma_only'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='pos'))
df_sample['lemma_con_aspect_sentiments'] = df_sample['cons_clean_lemma_only'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='neg'))
df_sample['Stopword_pro_aspect_sentiments'] = df_sample['pros_clean_with_stopwords'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='pos'))
df_sample['Stopword_con_aspect_sentiments'] = df_sample['cons_clean_with_stopwords'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='neg'))
df_sample['Symspell_pro_aspect_sentiments'] = df_sample['pros_clean_with_symspell'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='pos'))
df_sample['Symspell_con_aspect_sentiments'] = df_sample['cons_clean_with_symspell'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='neg'))

  0%|          | 0/761 [00:00<?, ?it/s]

  0%|          | 0/761 [00:00<?, ?it/s]

  0%|          | 0/761 [00:00<?, ?it/s]

  0%|          | 0/761 [00:00<?, ?it/s]

  0%|          | 0/761 [00:00<?, ?it/s]

  0%|          | 0/761 [00:00<?, ?it/s]

  0%|          | 0/761 [00:00<?, ?it/s]

  0%|          | 0/761 [00:00<?, ?it/s]

In [134]:
df_sample.to_csv('comparison_cleans.csv')

In [135]:
df_sample[['firm', 'pros', 'cons',
       'pros_clean_min', 'pros_clean_lemma_only',
       'pros_clean_with_stopwords', 'pros_clean_with_symspell',
       'cons_clean_min', 'cons_clean_lemma_only', 'cons_clean_with_stopwords',
       'cons_clean_with_symspell', 'min_pro_aspect_sentiments','lemma_pro_aspect_sentiments','Stopword_pro_aspect_sentiments','Symspell_pro_aspect_sentiments',
       'min_con_aspect_sentiments', 
       'lemma_con_aspect_sentiments', 
       'Stopword_con_aspect_sentiments', 
       'Symspell_con_aspect_sentiments']].to_csv('comparison_cleans.csv')

In [136]:
from collections import defaultdict

# Define clean types and columns
clean_types = ['min', 'lemma', 'Stopword', 'Symspell']
pro_cols = [f'{c}_pro_aspect_sentiments' for c in clean_types]
con_cols = [f'{c}_con_aspect_sentiments' for c in clean_types]

# Collect all aspects seen in the data
all_aspects = sorted({
    aspect
    for col in pro_cols + con_cols
    for entry in df_sample[col].dropna()
    for aspect in entry.keys()  # no need to eval, it's already a dict
})

# Count mentions in pros
pro_counts = defaultdict(dict)
for method in clean_types:
    col = f'{method}_pro_aspect_sentiments'
    for aspect in all_aspects:
        pro_counts[aspect][method] = df_sample[col].dropna().apply(lambda x: aspect in x).sum()

# Count mentions in cons
con_counts = defaultdict(dict)
for method in clean_types:
    col = f'{method}_con_aspect_sentiments'
    for aspect in all_aspects:
        con_counts[aspect][method] = df_sample[col].dropna().apply(lambda x: aspect in x).sum()

# Create DataFrames
pro_df = pd.DataFrame.from_dict(pro_counts, orient='index').fillna(0).astype(int).sort_index()
con_df = pd.DataFrame.from_dict(con_counts, orient='index').fillna(0).astype(int).sort_index()

# Pretty print like your example
print("\nAspect Mentions in PROS:")
print(f"{'Aspect':30}" + "".join([f"{ct:>12}" for ct in clean_types]))
print("-" * (30 + 12 * len(clean_types)))
for aspect, row in pro_df.iterrows():
    print(f"{aspect:30}" + "".join([f"{row[ct]:12}" for ct in clean_types]))

print("\nAspect Mentions in CONS:")
print(f"{'Aspect':30}" + "".join([f"{ct:>12}" for ct in clean_types]))
print("-" * (30 + 12 * len(clean_types)))
for aspect, row in con_df.iterrows():
    print(f"{aspect:30}" + "".join([f"{row[ct]:12}" for ct in clean_types]))



Aspect Mentions in PROS:
Aspect                                 min       lemma    Stopword    Symspell
------------------------------------------------------------------------------
culture                                151         149         149         149
diversity & inclusion                   14          14          14          14
growth & learning                      199         198         198         198
hr & recruitment                       135         132         128          83
job role clarity                        27          28          28          28
job security                            13          13          13          13
management                              81          64          64          64
office & location                       80          72          72          71
pay                                    151         143         143         143
people                                 208         170         170         170
performance feedback      

## Manual verification
After applying the 4 different cleans we reviewed 100 reviews ourselves to performerm our own aspect based sentiment analsysis. We also used an LLM to get a third opinion. 
We then compared the manual detection to the pyabsa applied to the 4 different cleans to determine which clean produces the accurate results

In [137]:
dfManual = pd.read_csv("Manual aspect detection.csv")

Parsing through the json like string from the pyabsa on 4 cleans

In [138]:
def parse_machine(cell: str) -> dict[str, int]:
    """
    Turn the JSON-ish string from the model into {aspect: label}.
    Empty/malformed → {}.
    """
    try:
        d = ast.literal_eval(cell)
        return {k: int(v["label"]) for k, v in d.items() if "label" in v}
    except Exception:
        return {}

_hum_re = re.compile(r"\s*([^:;]+)\s*:\s*([1-5])")

def parse_human(cell) -> dict[str, int]:
    """
    Turn 'culture:4; pay:2' into {'culture':4, 'pay':2}.
    Safely skip non-string (NaN) cells.
    """
    if not isinstance(cell, str):
        return {}
    return {m.group(1).strip(): int(m.group(2)) for m in _hum_re.finditer(cell)}



In [139]:
CLEAN_VARIANTS = {
    "min"      : ("min_pro_aspect_sentiments",      "min_con_aspect_sentiments"),
    "lemma"    : ("lemma_pro_aspect_sentiments",    "lemma_con_aspect_sentiments"),
    "stopword" : ("Stopword_pro_aspect_sentiments", "Stopword_con_aspect_sentiments"),
    "symspell" : ("Symspell_pro_aspect_sentiments", "Symspell_con_aspect_sentiments"),
}

In [140]:
def merge_reviews(*dicts):
    combined = {}
    keys = set().union(*[d.keys() for d in dicts])
    for key in keys:
        scores = [d.get(key) for d in dicts if d.get(key) is not None]
        if scores:
            combined[key] = round(sum(scores) / len(scores))
    return combined


Testing gold pro when we don't include chat

In [141]:
records = []

for idx, row in dfManual.iterrows():
    gold_marie_pro = parse_human(row["Marie's pros"])
    gold_marie_con = parse_human(row["Marie's pros"])
    gold_javi_pro  = parse_human(row["Javi's pros"])
    gold_javi_con  = parse_human(row["Javi's pros"])

    # Option A: Human only
    gold_pro = merge_reviews(gold_marie_pro, gold_javi_pro)
    gold_con = merge_reviews(gold_marie_con, gold_javi_con)


    gold_full = {**gold_pro, **gold_con}          # treat pros & cons together

    for variant, (pro_col, con_col) in CLEAN_VARIANTS.items():
        pred_pro  = parse_machine(row[pro_col])
        pred_con  = parse_machine(row[con_col])
        pred_full = {**pred_pro, **pred_con}

        aspects = set(gold_full) | set(pred_full)
        for asp in aspects:
            records.append(
                dict(method=variant,
                     aspect=asp,
                     gold = gold_full.get(asp),     # None if missing
                     pred = pred_full.get(asp))
            )

long = pd.DataFrame.from_records(records)
# drop rows where the human reviewer had no opinion for that aspect
long = long[long.gold.notna()]
long["gold_binary"] = 1
long["pred_binary"] = long["pred"].notna().astype(int)



In [142]:
METRICS = {}

for variant, grp in long.groupby("method"):
    y_true = grp["gold"]
    y_pred = grp["pred"].fillna(0)          # 0 = “model didn’t predict this aspect”

    # The class list forces metrics to treat all 5 sentiment levels equally
    labels = [1, 2, 3, 4, 5]

    macro_f1 = f1_score(y_true, y_pred, labels=labels, average="macro")
    micro_f1 = f1_score(y_true, y_pred, labels=labels, average="micro")
    acc      = accuracy_score(y_true, y_pred)

    METRICS[variant] = dict(macro_f1=macro_f1,
                            micro_f1=micro_f1,
                            exact_match=acc,
                            samples=len(grp))

results = (pd.DataFrame(METRICS)
           .T.sort_values("macro_f1", ascending=False)
           .round(3))


In [143]:
from sklearn.metrics import precision_score, recall_score, f1_score

aspect_metrics = {}

for variant, grp in long.groupby("method"):
    prec = precision_score(grp["gold_binary"], grp["pred_binary"])
    rec  = recall_score(grp["gold_binary"], grp["pred_binary"])
    f1   = f1_score(grp["gold_binary"], grp["pred_binary"])

    aspect_metrics[variant] = {
        "aspect_precision": round(prec, 3),
        "aspect_recall": round(rec, 3),
        "aspect_f1": round(f1, 3)
    }

aspect_results = pd.DataFrame(aspect_metrics).T


In [144]:
combined_results = results.join(aspect_results)
print("excluding chat")
print(combined_results.sort_values("macro_f1", ascending=False))


excluding chat
          macro_f1  micro_f1  exact_match  samples  aspect_precision  \
min          0.220     0.462        0.392    245.0               1.0   
symspell     0.217     0.481        0.392    245.0               1.0   
lemma        0.214     0.489        0.404    245.0               1.0   
stopword     0.190     0.490        0.404    245.0               1.0   

          aspect_recall  aspect_f1  
min               0.698      0.822  
symspell          0.629      0.772  
lemma             0.653      0.790  
stopword          0.649      0.787  


Testing gold pro including chat

In [145]:
records = []

for idx, row in dfManual.iterrows():
    gold_marie_pro = parse_human(row["Marie's pros"])
    gold_marie_con = parse_human(row["Marie's pros"])
    gold_javi_pro  = parse_human(row["Javi's pros"])
    gold_javi_con  = parse_human(row["Javi's pros"])
    gold_chat_pro  = parse_human(row["Chat's pros"])
    gold_chat_con  = parse_human(row["Chat's pros"])

    # Option A: Human only
    gold_pro = merge_reviews(gold_marie_pro, gold_javi_pro, gold_chat_pro)
    gold_con = merge_reviews(gold_marie_con, gold_javi_con, gold_chat_con)


    gold_full = {**gold_pro, **gold_con}          # treat pros & cons together

    for variant, (pro_col, con_col) in CLEAN_VARIANTS.items():
        pred_pro  = parse_machine(row[pro_col])
        pred_con  = parse_machine(row[con_col])
        pred_full = {**pred_pro, **pred_con}

        aspects = set(gold_full) | set(pred_full)
        for asp in aspects:
            records.append(
                dict(method=variant,
                     aspect=asp,
                     gold = gold_full.get(asp),     # None if missing
                     pred = pred_full.get(asp))
            )

long = pd.DataFrame.from_records(records)
# drop rows where the human reviewer had no opinion for that aspect
long = long[long.gold.notna()]
long["gold_binary"] = 1
long["pred_binary"] = long["pred"].notna().astype(int)



In [146]:
METRICS = {}

for variant, grp in long.groupby("method"):
    y_true = grp["gold"]
    y_pred = grp["pred"].fillna(0)          # 0 = “model didn’t predict this aspect”

    # The class list forces metrics to treat all 5 sentiment levels equally
    labels = [1, 2, 3, 4, 5]

    macro_f1 = f1_score(y_true, y_pred, labels=labels, average="macro")
    micro_f1 = f1_score(y_true, y_pred, labels=labels, average="micro")
    acc      = accuracy_score(y_true, y_pred)

    METRICS[variant] = dict(macro_f1=macro_f1,
                            micro_f1=micro_f1,
                            exact_match=acc,
                            samples=len(grp))

results = (pd.DataFrame(METRICS)
           .T.sort_values("macro_f1", ascending=False)
           .round(3))


In [147]:
from sklearn.metrics import precision_score, recall_score, f1_score

aspect_metrics = {}

for variant, grp in long.groupby("method"):
    prec = precision_score(grp["gold_binary"], grp["pred_binary"])
    rec  = recall_score(grp["gold_binary"], grp["pred_binary"])
    f1   = f1_score(grp["gold_binary"], grp["pred_binary"])

    aspect_metrics[variant] = {
        "aspect_precision": round(prec, 3),
        "aspect_recall": round(rec, 3),
        "aspect_f1": round(f1, 3)
    }

aspect_results = pd.DataFrame(aspect_metrics).T


In [148]:
combined_results = results.join(aspect_results)
print("including chat")
print(combined_results.sort_values("macro_f1", ascending=False))


including chat
          macro_f1  micro_f1  exact_match  samples  aspect_precision  \
min          0.188     0.362        0.290    314.0               1.0   
stopword     0.177     0.378        0.293    314.0               1.0   
symspell     0.175     0.365        0.280    314.0               1.0   
lemma        0.169     0.380        0.296    314.0               1.0   

          aspect_recall  aspect_f1  
min               0.602      0.751  
stopword          0.551      0.710  
symspell          0.535      0.697  
lemma             0.561      0.718  


## 📊 (for your reference not for final) Metrics explained
- ✅ Macro F1
    - Average F1 across all sentiment classes (1–5), giving equal weight to each class, no matter how common or rare.
    - Good when you care about rare classes (e.g., if 1 and 5 are underrepresented).
- ✅ Micro F1
    - F1 computed across all predictions, weighted by the number of instances (i.e., majority classes dominate).
    - Good overall performance summary when you care about total correct predictions.
- ✅ Exact Match
    - Percentage of predictions where the aspect and its sentiment rating exactly match the human gold.
    - Strict: both the right aspect and the right number.
- ✅ Aspect Precision
    - Of the aspects the model predicted, what fraction were actually in the human-reviewed set (regardless of sentiment score).
    - Measures “hallucination control.”
- ✅ Aspect Recall
    - Of the aspects humans labeled, what fraction did the model correctly detect (regardless of sentiment score).
    - Measures “coverage.”
- ✅ Aspect F1
    - Harmonic mean of aspect precision and recall.
    - Balances both coverage and correctness in detecting aspects (ignores score).

## Going forward, will be using the pro_clean_min and con_clean_min in the analysis

### Rerunning on the full sample

### Clean

In [149]:
dfFinal = pd.read_csv('filtered_england_recent.csv')

In [150]:
for col in ['pros', 'cons']:
    print(f"Processing column: {col}")
    dfFinal[f'{col}_clean_min'] = dfFinal[col].progress_apply(clean_min)

Processing column: pros


  0%|          | 0/76126 [00:00<?, ?it/s]

Processing column: cons


  0%|          | 0/76126 [00:00<?, ?it/s]

In [151]:
dfFinal.to_csv("preprocessed_england_dataset.csv", index=False)

### Applying aspet groups

In [152]:
dfFinal['min_pro_aspect_sentiments'] = dfFinal['pros_clean_min'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='pos'))
dfFinal['min_con_aspect_sentiments'] = dfFinal['cons_clean_min'].progress_apply(lambda x: analyze_all_aspect_groups(x, modified_aspect_groups, tone='neg'))

  0%|          | 0/76126 [00:00<?, ?it/s]

  0%|          | 0/76126 [00:00<?, ?it/s]

### Moving to aggregating the aspects per company

In [153]:
def flatten_aspect_sentiments(row, source):
    results = []
    firm = row.get('firm') or row.get('company')  # fallback
    sentiments = row.get(source, {})
    for aspect, scores in sentiments.items():
        # Aggregate actual star rating (1–5)
        rating = scores.get("label")
        all_stars = {i: scores.get(i, 0.0) for i in range(1, 6)}

        results.append({
            "firm": firm,
            "aspect": aspect,
            "source": source.replace('_aspect_sentiments', ''),  # 'pro' or 'con'
            "star_rating": rating,
            **all_stars
        })
    return results

In [154]:
# 3. Apply to full dataset
flattened = []
for _, row in dfFinal.iterrows():
    flattened += flatten_aspect_sentiments(row, 'min_pro_aspect_sentiments')
    flattened += flatten_aspect_sentiments(row, 'min_con_aspect_sentiments')

flat_df = pd.DataFrame(flattened)

# 4. Aggregate star ratings + count mentions per company/aspect
company_aspect_matrix = flat_df.groupby(['firm', 'aspect'])['star_rating'].agg(
    avg_star_rating='mean',
    n_mentions='count'
).reset_index()
company_aspect_pivot = company_aspect_matrix.pivot(
    index='firm',
    columns='aspect',
    values='avg_star_rating'
)

In [155]:
company_aspect_matrix

,firm,aspect,avg_star_rating,n_mentions
0,AFH Wealth Management,culture,2.769231,13
1,AFH Wealth Management,growth & learning,3.444444,9
2,AFH Wealth Management,hr & recruitment,2.521739,23
3,AFH Wealth Management,job role clarity,1.000000,4
4,AFH Wealth Management,job security,1.400000,5
...,...,...,...,...
4613,the LEGO Group,remote work,3.000000,3
4614,the LEGO Group,stress & pressure,2.700000,10
4615,the LEGO Group,teamwork & collaboration,3.538462,39
4616,the LEGO Group,tools & resources,3.428571,14


In [156]:
company_aspect_pivot

aspect,culture,diversity & inclusion,growth & learning,hr & recruitment,job role clarity,job security,management,office & location,pay,people,performance feedback,perks & benefits,remote work,stress & pressure,teamwork & collaboration,tools & resources,work-life balance
firm,,,,,,,,,,,,,,,,,
AFH Wealth Management,2.769231,NaN,3.444444,2.521739,1.000000,1.400000,2.375000,3.142857,1.894737,3.294118,2.500000,3.000000,NaN,2.000000,2.571429,2.111111,3.166667
AJ Bell,3.416667,3.000000,3.250000,3.000000,2.125000,2.800000,2.804878,3.468085,2.428571,3.438596,2.875000,2.862745,4.333333,2.928571,3.290323,3.413793,2.870968
ASOS,3.209302,2.944444,2.550360,2.807692,2.350877,1.400000,1.754464,3.285714,2.335000,3.311787,1.809524,3.685237,3.000000,2.404040,2.701987,2.864407,2.644444
AXA UK,3.543860,4.250000,3.534091,3.192982,3.100000,2.500000,2.781250,3.057143,3.029851,3.704225,2.333333,3.953125,3.125000,1.800000,3.702703,3.454545,3.730769
Abcam,4.153846,4.250000,3.474576,3.513514,3.266667,2.500000,3.530612,3.302632,2.418605,4.119048,3.400000,3.725000,4.750000,3.119048,4.105263,3.833333,4.063830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XPS Pensions Group,4.000000,NaN,3.684211,3.333333,4.000000,2.500000,3.200000,3.090909,2.000000,4.333333,1.750000,3.125000,NaN,2.000000,4.222222,3.666667,2.454545
YOOX NET A PORTER GROUP,2.450000,3.333333,2.151163,2.775000,1.700000,1.333333,1.634146,3.275000,1.871429,3.048780,1.615385,3.530435,3.800000,2.041667,2.846154,2.693878,2.821429
i Net Solution,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,2.500000,4.000000,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Exporting results

In [157]:
# 6. Export
company_aspect_matrix.to_csv('Final_company_aspect_matrix_with_counts.csv', index=False)
company_aspect_pivot.to_csv('Final_company_aspect_matrix_wide.csv', index=False)
